<a href="https://colab.research.google.com/github/kutouxiyiji/test1/blob/master/Pytorch_try_basic_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Try from Numpy. 

In [0]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms


# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189)

In [0]:
# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 


# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)  # w convert shape(10,3) to (10,2)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())



tensor(2.)
tensor(1.)
tensor(1.)
w:  Parameter containing:
tensor([[-0.4245,  0.3256, -0.1535],
        [-0.1245, -0.1420,  0.1049]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.3081, -0.2722], requires_grad=True)
loss:  1.3071843385696411
dL/dw:  tensor([[-0.6774,  0.9695, -0.4858],
        [ 0.0058, -0.2372,  0.0654]])
dL/db:  tensor([ 0.4178, -0.2284])
loss after 1 step optimization:  1.2880946397781372


In [0]:
loss.item()

1.2880946397781372

In [0]:
for i in range(30):
  optimizer.step()
  pred = linear(x)
  loss = criterion(pred,y)
  loss.backward()
  print('loss after {} step'.format(i), loss.item())
  

loss after 0 step 0.8714390397071838
loss after 1 step 0.8656208515167236
loss after 2 step 0.8670972585678101
loss after 3 step 0.8750539422035217
loss after 4 step 0.888118326663971
loss after 5 step 0.9044754505157471
loss after 6 step 0.9220167398452759
loss after 7 step 0.9385094046592712
loss after 8 step 0.9517755508422852
loss after 9 step 0.959868311882019
loss after 10 step 0.9612309336662292
loss after 11 step 0.9548288583755493
loss after 12 step 0.9402453899383545
loss after 13 step 0.9177317023277283
loss after 14 step 0.888209879398346
loss after 15 step 0.853226363658905
loss after 16 step 0.8148592114448547
loss after 17 step 0.775585412979126
loss after 18 step 0.738115668296814
loss after 19 step 0.7052093744277954
loss after 20 step 0.6794812083244324
loss after 21 step 0.6632124185562134
loss after 22 step 0.6581805348396301
loss after 23 step 0.6655189990997314
loss after 24 step 0.6856166124343872
loss after 25 step 0.7180643677711487
loss after 26 step 0.7616546

In [0]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()


# ================================================================== #
#                         4. Input pipline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

0it [00:00, ?it/s]

100%|█████████▉| 170156032/170498071 [00:23<00:00, 5377321.91it/s]

torch.Size([3, 32, 32])
6


In [0]:
# ================================================================== #
#                5. Input pipline for custom dataset                 #
# ================================================================== #

# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

ValueError: ignored

In [0]:

# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)


# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth

100%|██████████| 46827520/46827520 [00:00<00:00, 79355381.52it/s]

torch.Size([64, 100])


IncompatibleKeys(missing_keys=[], unexpected_keys=[])